In [11]:
!pip install --upgrade pip setuptools wheel


In [12]:
!pip install fasttext-wheel



In [73]:
import fasttext
import pandas as pd
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [14]:
df = pd.read_csv("processed.csv") 
df.head() 

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,tokens
0,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses,"['high', 'hopes', 'wanted', 'work', 'initially..."
1,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants,"['jumpsuit', 'fun', 'flirty', 'fabulous', 'tim..."
2,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses,"['shirt', 'due', 'adjustable', 'front', 'tie',..."
3,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses,"['tracy', 'reese', 'dresses', 'petite', 'feet'..."
4,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits,"['basket', 'hte', 'person', 'store', 'pick', '..."


In [15]:
print(type(df['tokens'].iloc[0]))


<class 'str'>


In [16]:
with open("corpus.txt", "w", encoding="utf-8") as f:
    for tokens_str in df['tokens']:
        tokens_list = ast.literal_eval(tokens_str)
        f.write(" ".join(tokens_list) + "\n")

In [17]:
model = fasttext.train_unsupervised("corpus.txt", model="skipgram")
model.save_model("fasttext_model.bin")

In [18]:
sentence = df['tokens'].iloc[0]
print(model.get_sentence_vector(sentence))

[-0.0502181   0.11608545 -0.05071434 -0.04794019  0.01976476 -0.00244919
 -0.03864212  0.01786429  0.08201405 -0.0357217  -0.06928198  0.08953506
 -0.04521319  0.02605594  0.14996313  0.0282911   0.09404537 -0.09800475
  0.0633342   0.15720576  0.00523448  0.09462331 -0.06708309 -0.05246284
 -0.00136684  0.03533037 -0.08746415  0.04518271  0.09698678 -0.00834511
 -0.06946409 -0.06553639  0.06967368 -0.06657588 -0.07891813  0.04054167
  0.00988678 -0.05205761 -0.02959752 -0.07411259  0.10429993  0.11876374
  0.04700925  0.13201074  0.1778085   0.03452186  0.05291465  0.01296963
  0.01362985  0.0646021   0.03066067 -0.03603581  0.00462547 -0.01155513
 -0.08011226 -0.05649599  0.02718875 -0.05238556 -0.0117139   0.10366348
  0.11310197  0.02029812  0.05050264  0.09367456 -0.0125505   0.09244841
  0.00896423  0.08929329 -0.00904857 -0.06396701 -0.06490528  0.06724589
 -0.03019753 -0.06618586  0.00840686 -0.05807334  0.0116625  -0.02288625
  0.01348011  0.06306261 -0.03789584 -0.01199881 -0

In [19]:
def get_review_vector(tokens):
    # Join tokens into a sentence (fastText expects a string, not list)
    sentence = " ".join(tokens)
    return model.get_sentence_vector(sentence)

# Apply to DataFrame
df["vector"] = df["tokens"].apply(get_review_vector)

df

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,tokens,vector
0,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses,"['high', 'hopes', 'wanted', 'work', 'initially...","[0.057855498, 0.029851526, 0.054970015, -0.004..."
1,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants,"['jumpsuit', 'fun', 'flirty', 'fabulous', 'tim...","[0.044263314, 0.013135654, 0.016781278, -0.009..."
2,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses,"['shirt', 'due', 'adjustable', 'front', 'tie',...","[0.056948345, 0.015471387, 0.047482353, -0.008..."
3,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses,"['tracy', 'reese', 'dresses', 'petite', 'feet'...","[0.06264694, 0.018949205, 0.04784146, 0.000337..."
4,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits,"['basket', 'hte', 'person', 'store', 'pick', '...","[0.05174453, 0.029474493, 0.052665066, -0.0098..."
...,...,...,...,...,...,...,...,...,...,...,...,...
19657,1104,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General Petite,Dresses,Dresses,"['happy', 'snag', 'price', 'easy', 'slip', 'cu...","[0.04700138, 0.039847236, 0.032771066, 0.00325..."
19658,862,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0,General Petite,Tops,Knits,"['reminds', 'maternity', 'clothes', 'stretchy'...","[0.056523368, 0.016383236, 0.0423402, 0.002749..."
19659,1104,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1,General Petite,Dresses,Dresses,"['worked', 'glad', 'store', 'order', 'online']","[0.078630425, 0.026951166, 0.038689002, 0.0039..."
19660,1084,28,"Very cute dress, perfect for summer parties an...",I bought this dress for a wedding i have this ...,3,1,2,General,Dresses,Dresses,"['wedding', 'summer', 'medium', 'fits', 'waist...","[0.05940797, 0.015582375, 0.045298353, -0.0102..."


In [ ]:

with open("corpus.txt", "r", encoding="utf-8") as f:
    corpus = f.read()
    sentences = []
    for line in corpus.split("\n"):
        sentences.append(line)

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)



Shape: (19663, 7195)
Non-zeros: 335639
Density: 0.0023724214444947045


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 28 stored elements and shape (1, 7195)>

array([0.10510255, 0.16844688, 0.10688757, 0.08943434, 0.17097754,
       0.17994811, 0.1109029 , 0.11017318, 0.25678866, 0.14687211,
       0.15791012, 0.20116306, 0.0904633 , 0.29992434, 0.11400558,
       0.10491448, 0.09944862, 0.273902  , 0.1383493 , 0.48677407,
       0.16401783, 0.19309613, 0.19468663, 0.09819656, 0.18693506,
       0.15731157, 0.22041891, 0.13796907])